In [ ]:
file_content = """A,T,C,G
GCAT
2 2 2 0 1 2 1 2"""

with open("input1.txt", "w") as f:
    f.write(file_content)

In [ ]:
def input_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    pool = lines[0].strip().split(',')
    target = lines[1].strip()
    sid_digits = list(map(int, lines[2].strip().split()))
    return pool, target, sid_digits

def get_weights(sid_digits, length):
    return sid_digits[-length:]

def utility(gene, target, weights):
    max_len = max(len(gene), len(target))
    score = 0

    for i in range(max_len):
        if i < len(gene):
            gen = ord(gene[i])
        else:
            gen = 0

        if i < len(target):
            tar = ord(target[i])
        else:
            tar = 0

        if i < len(weights):
            wi = weights[i]
        else:
            wi = 1

        difference = abs(gen - tar)
        weighted_difference = wi * difference

        score = score + weighted_difference

    return -score


def minimax(pool, gene, maximizing, target, weights, alpha, beta):
    if not pool or len(gene) >= len(target):
        utility_score = utility(gene, target, weights)
        return utility_score, gene

    if maximizing:
        max_eval = float('-inf')
        best_seq = None

        for i, n in enumerate(pool):
            new_pool = pool[:i] + pool[i+1:]
            new_gene = gene + n

            eval_score, seq = minimax(new_pool, new_gene, False, target, weights, alpha, beta)              # Recursive call:minimizing

            if eval_score > max_eval:
                max_eval = eval_score
                best_seq = seq

            if alpha < eval_score:
                alpha = eval_score

            if beta <= alpha:
                break

        return max_eval, best_seq

    else:
        min_eval = float('inf')
        best_seq = None

        for i, n in enumerate(pool):
            new_pool = pool[:i] + pool[i+1:]
            new_gene = gene + n

            eval_score, seq = minimax(new_pool, new_gene, True, target, weights, alpha, beta)

            if eval_score < min_eval:
                min_eval = eval_score
                best_seq = seq

            if beta > eval_score:
                beta = eval_score

            if beta <= alpha:
                break

        return min_eval, best_seq


def main():
    pool, target, sid_digits = input_file('input1.txt')
    weights = get_weights(sid_digits, len(target))
    score, sequence = minimax(pool, "", True, target, weights, float('-inf'), float('inf'))
    print('Best gene sequence generated:', sequence)
    print('Utility score:', score)

if __name__ == "__main__":
    main()


Best gene sequence generated: TGAC
Utility score: -55


In [ ]:
file_content = """A,T,C,G,S
GCAT
2 2 2 0 1 2 1 2

"""
with open("input2.txt", "w") as f:
    f.write(file_content)

In [ ]:
def input_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    pool = lines[0].strip().split(',')
    target = lines[1].strip()
    sid_digits = list(map(int, lines[2].strip().split()))
    return pool, target, sid_digits

def get_weights(sid_digits, length):
    return sid_digits[-length:]

def apply_genetic_booster(weights, s_position, sid_digits):
    if s_position == -1:  # No 'S' in sequence
        return weights
    booster = (sid_digits[0] * 10 + sid_digits[1]) / 100
    new_weights = weights.copy()
    for i in range(s_position, len(new_weights)):
        new_weights[i] *= booster
    return new_weights

def utility(gene, target, weights, s_position, sid_digits):
    weights = apply_genetic_booster(weights, s_position, sid_digits)
    max_len = max(len(gene), len(target))
    score = 0

    for i in range(max_len):
        gen = ord(gene[i]) if i < len(gene) else 0
        tar = ord(target[i]) if i < len(target) else 0
        wi = weights[i] if i < len(weights) else 1
        difference = abs(gen - tar)
        weighted_difference = wi * difference
        score += weighted_difference

    return -score

def minimax(pool, gene, maximizing, target, weights, sid_digits, alpha, beta, use_s=False, max_length=None):
    if not pool or len(gene) >= (max_length or len(target)):
        s_position = gene.find('S') if 'S' in gene else -1
        if use_s and s_position == -1:
            return float('-inf') if maximizing else float('inf'), gene
        utility_score = utility(gene, target, weights, s_position, sid_digits)
        return utility_score, gene

    if maximizing:
        max_eval = float('-inf')
        best_seq = None

        candidates = ['S'] + [n for n in pool if n != 'S'] if use_s and 'S' in pool else pool
        for n in candidates:
            if n not in pool:
                continue
            i = pool.index(n)
            new_pool = pool[:i] + pool[i+1:]
            new_gene = gene + n
            eval_score, seq = minimax(new_pool, new_gene, False, target, weights, sid_digits, alpha, beta, use_s, max_length)

            if eval_score > max_eval:
                max_eval = eval_score
                best_seq = seq

            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break

        return max_eval, best_seq

    else:
        min_eval = float('inf')
        best_seq = None

        for i, n in enumerate(pool):
            new_pool = pool[:i] + pool[i+1:]
            new_gene = gene + n
            eval_score, seq = minimax(new_pool, new_gene, True, target, weights, sid_digits, alpha, beta, use_s, max_length)

            if eval_score < min_eval:
                min_eval = eval_score
                best_seq = seq

            beta = min(beta, eval_score)
            if beta <= alpha:
                break

        return min_eval, best_seq

def main():
    pool, target, sid_digits = input_file('input2.txt')
    weights = get_weights(sid_digits, len(target))

    pool_without_s = [n for n in pool if n != 'S']
    score_without_s, _ = minimax(pool_without_s, "", True, target, weights, sid_digits, float('-inf'), float('inf'), use_s=False, max_length=len(target))

    max_length = len(target) + 1 if 'S' in pool else len(target)
    score_with_s, sequence_with_s = minimax(pool, "", True, target, weights, sid_digits, float('-inf'), float('inf'), use_s=('S' in pool), max_length=max_length)

    if 'S' in pool and score_with_s > score_without_s:
        print("YES")
    else:
        print("NO")
    print(f"With special nucleotide")
    print(f"Best gene sequence generated: {sequence_with_s}, Utility score: {score_with_s:.2f}")

if __name__ == "__main__":
    main()

NO
With special nucleotide
Best gene sequence generated: STGAC, Utility score: -86.80
